# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [51]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
weather_csv = pd.read_csv('../WeatherPy/Output/city_weather_data.csv')
del weather_csv['Unnamed: 0']
weather_csv

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Puerto Ayora,-0.7393,-90.3518,74.55,86,41,7.00,EC,1622332964
1,Mogadishu,2.0371,45.3438,78.17,84,82,12.64,SO,1622332964
2,Asău,46.4333,26.4000,54.52,99,87,1.90,RO,1622332964
3,Busselton,-33.6500,115.3333,53.10,84,95,19.75,AU,1622332965
4,Rojhan,28.6868,69.9534,91.92,44,0,11.16,PK,1622332965
...,...,...,...,...,...,...,...,...,...
555,Xapuri,-10.6517,-68.5044,79.97,67,75,2.93,BR,1622333131
556,Kiryat Gat,31.6100,34.7642,68.68,86,1,1.25,IL,1622333131
557,La Joya,18.6667,-99.4667,86.40,40,87,5.08,MX,1622333132
558,Saint-Louis,16.3333,-15.0000,76.55,50,4,14.38,SN,1622333132


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key=g_key)
locations = weather_csv[["Lat", "Lng"]]
humidity = weather_csv['Humidity']

In [18]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
ideal_weather = weather_csv[(weather_csv['Max Temp'] <= 80) & (weather_csv['Max Temp'] >= 70)]
ideal_weather = ideal_weather[(ideal_weather['Cloudiness'] == 0)] 
ideal_weather = ideal_weather[(ideal_weather['Wind Speed'] <= 10)] 
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
307,Makat,47.6469,53.3463,71.73,58,0,4.29,KZ,1622333057
328,Dzaoudzi,-12.7887,45.2699,78.69,73,0,9.22,YT,1622333063
470,Rafsanjān,30.4067,55.9939,77.31,11,0,5.84,IR,1622333107


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = ideal_weather
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
307,Makat,47.6469,53.3463,71.73,58,0,4.29,KZ,1622333057,
328,Dzaoudzi,-12.7887,45.2699,78.69,73,0,9.22,YT,1622333063,
470,Rafsanjān,30.4067,55.9939,77.31,11,0,5.84,IR,1622333107,


In [56]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    try:
        params = {
            "location": f"{row['Lat']},{row['Lng']}", 
            "radius": 5000,
            "type": "hotel",
            "key": g_key,}
          

        response = requests.get(base_url, params=params).json()
        results = response['results']
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except IndexError:
        print("Missing field/result... skipping.")
hotel_df        

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
307,Makat,47.6469,53.3463,71.73,58,0,4.29,KZ,1622333057,Makat
328,Dzaoudzi,-12.7887,45.2699,78.69,73,0,9.22,YT,1622333063,Mamoudzou
470,Rafsanjān,30.4067,55.9939,77.31,11,0,5.84,IR,1622333107,Rafsanjan


In [58]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [64]:
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))